In [351]:
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Dataset

import pandas as pd
import numpy as np

from random import shuffle, seed

In [395]:
# Process original dataset, create windows (60 samples(rows), about 1 second)
first_df = pd.read_csv("./Activity Recognition from Single Chest-Mounted Accelerometer/3.csv")
first_df.columns = ["time", "x_acce", "y_acce", "z_acce", "label"]
first_df = first_df[59:]
data = []
# first_df
window = 1
while window*60 < len(first_df.index):
    data60 = first_df[(window - 1)*60:window*60]
    data.append(data60.values)
    window += 1

len(data)

1704

In [396]:
# delete window if multiple activities presents
cleaned_data = []
for i in data:
    previous_label = -1
    for j in i:
        current_label = j[4]
        if (previous_label != -1) and (current_label != previous_label):
            print("data contains different activities!")
            break
        else:
            previous_label = current_label
    else:
        cleaned_data.append(i)

data contains different activities!
data contains different activities!
data contains different activities!
data contains different activities!
data contains different activities!
data contains different activities!
data contains different activities!
data contains different activities!


In [397]:
len(cleaned_data)

1696

In [398]:
# shuffle the data
seed(101)
shuffle(cleaned_data)
#cleaned_data

In [399]:
# extract label from each window
labels = []
for i in cleaned_data:
    label = i[0][4]
    labels.append(label)
labels = np.array(labels)
#labels

In [400]:
# extract features from each window
features = []
for i in cleaned_data:
    new = np.delete(i, 4, 1)
    features.append(new)
features = np.array(features)
#features

In [401]:
# combine the features and labels, and convert it to a csv
k = list(zip(features, labels))
activity_data = pd.DataFrame(k)
#activity_data

In [402]:
# split the data to test and train
activity_data.columns = ["features", "labels"]
activity_data_train = activity_data[:int(len(activity_data)*0.7)]
activity_data_test = activity_data[int(len(activity_data)*0.7):]
activity_data_train.to_csv("./activity_data_train.csv", encoding='utf-8-sig')
activity_data_train.to_csv("./activity_data_test.csv", encoding='utf-8-sig')

In [403]:
# define our dataset in pytorch
class DatasetHAR(Dataset):
    
    def __init__(self, file, transform=None):
        #self.data = pd.read_csv(file_path)
        self.data = file
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        # load image as ndarray type (Height * Width * Channels)
        # be carefull for converting dtype to np.uint8 [Unsigned integer (0 to 255)]
        # in this example, i don't use ToTensor() method of torchvision.transforms
        # so you can convert numpy ndarray shape to tensor in PyTorch (H, W, C) --> (C, H, W)
        
        features = torch.tensor(self.data["features"].iloc[index])
        features = features.view(60, 1, 4)
        labels = torch.tensor(self.data["labels"].iloc[index], dtype=torch.long)
        #print(labels.type())
        
#         if self.transform is not None:
#             image = self.transform(image)
            
        return features, labels

In [404]:
# construct training and testing dataset in csv
# train_dataset = DatasetHAR("./activity_data_train.csv")
# test_dataset = DatasetHAR("./activity_data_test.csv")
train_dataset = DatasetHAR(activity_data_train)
test_dataset = DatasetHAR(activity_data_test)
feature, label = train_dataset.__getitem__(0)
#feature

In [405]:
# load data
trainloader = DataLoader(train_dataset, batch_size=64)
testloader = DataLoader(test_dataset, batch_size=64)

In [416]:
model = models.resnet18(pretrained=False)
model.conv1 = torch.nn.Conv2d(60, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
model.fc = torch.nn.Linear(512, 8, bias=True)
model = model.double()
model

ResNet(
  (conv1): Conv2d(60, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace)
      (conv2): C

In [417]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)

epochs = 30

train_losses, test_losses = [], []
for e in range(epochs):
    running_loss = 0
    for feature, label in trainloader:
        
        optimizer.zero_grad()
        
        log_ps = model(feature)
        loss = criterion(log_ps, label)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        #print(running_loss)
        
    else:
        test_loss = 0
        accuracy = 0
        
        # Turn off gradients for validation, saves memory and computations
        with torch.no_grad():
            for feature, label in testloader:
                log_ps = model(feature)
                test_loss += criterion(log_ps, label)
                
                ps = torch.exp(log_ps)
                top_p, top_class = ps.topk(1, dim=1)
                equals = top_class == label.view(*top_class.shape)
                accuracy += torch.mean(equals.type(torch.FloatTensor))
                
        train_losses.append(running_loss/len(trainloader))
        test_losses.append(test_loss/len(testloader))

        print("Epoch: {}/{}.. ".format(e+1, epochs),
              "Training Loss: {:.3f}.. ".format(running_loss/len(trainloader)),
              "Test Loss: {:.3f}.. ".format(test_loss/len(testloader)),
              "Test Accuracy: {:.3f}".format(accuracy/len(testloader)))

Epoch: 1/30..  Training Loss: 0.859..  Test Loss: 0.371..  Test Accuracy: 0.866
Epoch: 2/30..  Training Loss: 0.461..  Test Loss: 0.373..  Test Accuracy: 0.873
Epoch: 3/30..  Training Loss: 0.421..  Test Loss: 0.386..  Test Accuracy: 0.847
Epoch: 4/30..  Training Loss: 0.400..  Test Loss: 0.312..  Test Accuracy: 0.882
Epoch: 5/30..  Training Loss: 0.371..  Test Loss: 0.340..  Test Accuracy: 0.869
Epoch: 6/30..  Training Loss: 0.350..  Test Loss: 0.332..  Test Accuracy: 0.861
Epoch: 7/30..  Training Loss: 0.361..  Test Loss: 0.356..  Test Accuracy: 0.853
Epoch: 8/30..  Training Loss: 0.344..  Test Loss: 0.278..  Test Accuracy: 0.898
Epoch: 9/30..  Training Loss: 0.335..  Test Loss: 0.331..  Test Accuracy: 0.855
Epoch: 10/30..  Training Loss: 0.323..  Test Loss: 0.306..  Test Accuracy: 0.869
Epoch: 11/30..  Training Loss: 0.330..  Test Loss: 0.279..  Test Accuracy: 0.894
Epoch: 12/30..  Training Loss: 0.328..  Test Loss: 0.303..  Test Accuracy: 0.865
Epoch: 13/30..  Training Loss: 0.293.

## The rest is using a simple linear NN to compare with the ResNet

In [369]:
# define our dataset in pytorch
class DatasetH(Dataset):
    
    def __init__(self, file, transform=None):
        #self.data = pd.read_csv(file_path)
        self.data = file
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        # load image as ndarray type (Height * Width * Channels)
        # be carefull for converting dtype to np.uint8 [Unsigned integer (0 to 255)]
        # in this example, i don't use ToTensor() method of torchvision.transforms
        # so you can convert numpy ndarray shape to tensor in PyTorch (H, W, C) --> (C, H, W)
        
        features = torch.tensor(self.data["features"].iloc[index])
        labels = torch.tensor(self.data["labels"].iloc[index], dtype=torch.long)
        #print(labels.type())
        
#         if self.transform is not None:
#             image = self.transform(image)
            
        return features, labels

In [370]:
train_dataset2 = DatasetH(activity_data_train)
test_dataset2 = DatasetH(activity_data_test)
feature, label = train_dataset.__getitem__(0)
feature.shape

torch.Size([60, 1, 4])

In [371]:
# load data
trainloader2 = DataLoader(train_dataset2, batch_size=64)
testloader2 = DataLoader(test_dataset2, batch_size=64)

In [379]:
model2 = nn.Sequential(nn.Linear(240, 128),
                      nn.ReLU(),
                      nn.Linear(128, 64),
                      nn.ReLU(),
                      nn.Linear(64, 8))
model2 = model2.double()

In [380]:
# Test
criterion2 = nn.CrossEntropyLoss()
optimizer2 = optim.Adam(model2.parameters(), lr=0.003)

epochs = 30

train_losses, test_losses = [], []
for e in range(epochs):
    running_loss = 0
    for feature, label in trainloader2:
        feature = feature.view(feature.shape[0], -1)
        optimizer2.zero_grad()
        
        log_ps = model2(feature)
        loss = criterion2(log_ps, label)
        loss.backward()
        optimizer2.step()
        
        running_loss += loss.item()
        #print(running_loss)
    else:
        test_loss = 0
        accuracy = 0

        # Turn off gradients for validation, saves memory and computations
        with torch.no_grad():
            for feature, label in testloader2:
                feature = feature.view(feature.shape[0], -1)
                #print(feature.shape)
                #print(feature.shape)
                #print(label.shape)
                log_ps = model2(feature)
                test_loss += criterion2(log_ps, label)

                ps = torch.exp(log_ps)
                top_p, top_class = ps.topk(1, dim=1)
                equals = top_class == label.view(*top_class.shape)
                accuracy += torch.mean(equals.type(torch.FloatTensor))

        train_losses.append(running_loss/len(trainloader2))
        test_losses.append(test_loss/len(testloader2))

        print("Epoch: {}/{}.. ".format(e+1, epochs),
              "Training Loss: {:.3f}.. ".format(running_loss/len(trainloader)),
              "Test Loss: {:.3f}.. ".format(test_loss/len(testloader)),
              "Test Accuracy: {:.3f}".format(accuracy/len(testloader)))

Epoch: 1/30..  Training Loss: 2183.701..  Test Loss: 457.619..  Test Accuracy: 0.240
Epoch: 2/30..  Training Loss: 248.914..  Test Loss: 164.841..  Test Accuracy: 0.240
Epoch: 3/30..  Training Loss: 112.447..  Test Loss: 102.499..  Test Accuracy: 0.240
Epoch: 4/30..  Training Loss: 69.074..  Test Loss: 81.733..  Test Accuracy: 0.240
Epoch: 5/30..  Training Loss: 53.706..  Test Loss: 69.147..  Test Accuracy: 0.265
Epoch: 6/30..  Training Loss: 29.002..  Test Loss: 91.389..  Test Accuracy: 0.309
Epoch: 7/30..  Training Loss: 49.801..  Test Loss: 36.282..  Test Accuracy: 0.455
Epoch: 8/30..  Training Loss: 22.436..  Test Loss: 13.556..  Test Accuracy: 0.547
Epoch: 9/30..  Training Loss: 9.847..  Test Loss: 11.244..  Test Accuracy: 0.552
Epoch: 10/30..  Training Loss: 6.211..  Test Loss: 7.810..  Test Accuracy: 0.643
Epoch: 11/30..  Training Loss: 4.657..  Test Loss: 9.455..  Test Accuracy: 0.702
Epoch: 12/30..  Training Loss: 5.076..  Test Loss: 3.149..  Test Accuracy: 0.762
Epoch: 13/30.